In [1]:
import config

import logging
import random
import numpy as np
import matplotlib.pyplot as plt

logger = logging.getLogger(__name__)

2023-05-15 15:41:32,669 - INFO     | config     | Loading environment variables


In [22]:
import glob
import pickle
import re

files = glob.glob('embeddings/*_test.pkl')
embeddings_data = {}

for file in files:
    logger.info(f'Loading {file}')
    match_obj = re.search(r'/(?P<output>[^_/]+)_', file)
    filename = match_obj.group('output')
    with open(file, 'rb') as fp:
        dict = pickle.load(fp)
        embeddings_data[filename] = dict

2023-05-15 15:53:23,379 - INFO     | __main__   | Loading embeddings/glove_embeddings_test.pkl
2023-05-15 15:53:23,381 - INFO     | __main__   | Loading embeddings/mpnet_embeddings_test.pkl
2023-05-15 15:53:23,382 - INFO     | __main__   | Loading embeddings/distil_embeddings_test.pkl
2023-05-15 15:53:23,383 - INFO     | __main__   | Loading embeddings/wiki_embeddings_test.pkl
2023-05-15 15:53:23,384 - INFO     | __main__   | Loading embeddings/w2v_embeddings_test.pkl


# Clustering

Testing on mpnet embeddings of Llama 7B output.

In [24]:
mpnet_llama = embeddings_data['mpnet']['llama_7b']

array([[ 0.04991404,  0.01245025, -0.03698337, ...,  0.05343255,
        -0.01261293, -0.00499435],
       [ 0.05668905,  0.01711162, -0.03224138, ...,  0.02460312,
        -0.02274811, -0.01416241],
       [ 0.06095264, -0.03052419, -0.02791323, ...,  0.00879415,
        -0.06087885, -0.01180774],
       ...,
       [ 0.0707155 ,  0.00652098, -0.03405986, ...,  0.00563026,
        -0.04203464, -0.00584347],
       [ 0.0707408 ,  0.02420891, -0.04428411, ...,  0.01481363,
        -0.01738054, -0.0160687 ],
       [ 0.08725643,  0.00544711, -0.05242467, ..., -0.007789  ,
        -0.06486614, -0.00444929]], dtype=float32)

In [24]:
# plotting
def plot_clusters(embeddings: list, clusters, n_neighbors, min_dist: float):
    umap_data = umap.UMAP(n_neighbors=n_neighbors, 
                          n_components=2, 
                          min_dist = min_dist,  
                          #metric='cosine',
                          random_state=42).fit_transform(embeddings)
    
    point_size = 100.0 / np.sqrt(embeddings.shape[0])
    
    result = pd.DataFrame(umap_data, columns=['x', 'y'])
    result['labels'] = clusters.labels_

    fig, ax = plt.subplots(figsize=(14, 8))
    outliers = result[result.labels == -1]
    clustered = result[result.labels != -1]
    plt.scatter(outliers.x, outliers.y, color = 'lightgrey', s=point_size)
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=point_size, cmap='jet')
    plt.colorbar()
    plt.show()

## UMAP + HDBSCAN

In [11]:
import umap
import hdbscan